In [1]:
from random import randint
import os
import numpy as np
from curvative import *
from util import *
#from pclpy import pcl
from keras.layers import *
from keras.activations import *
from keras.models import Model,load_model
from keras.optimizers import *
from keras import losses
import keras

import tensorflow as tf

from Loss import *
from keras import backend as K
from keras.callbacks import TensorBoard

/GPUFS/sysu_khuang_2/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/GPUFS/sysu_khuang_2/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/GPUFS/sysu_khuang_2/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/GPUFS/sysu_khuang_2/.local/lib/python3.6/site-packages/tensorflow/

In [2]:
config = tf.ConfigProto()
config.gpu_options.visible_device_list="0"
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)

In [29]:
traindata = np.load('../square-sample2.npy')

In [30]:
curdata = np.load('../square-curvature-sample2.npy')

In [31]:
train_xx = traindata
cur_xx = curdata

In [32]:
train_xx_tensor = np.reshape(train_xx,[-1,5,1568,32,3])
cur_xx_tensor = np.reshape(cur_xx,[-1,5,1568,32,1])

In [17]:
model = load_model('../square-curvature-pretrained.h5',custom_objects={'loss_tf':loss_tf,'tf':tf})

In [18]:
model.compile(Adam(lr=0.0003),loss=loss_tf)

In [33]:
feature_layer_model = Model(inputs=model.input,outputs=model.get_layer('time_distributed_2').output)
feature = feature_layer_model.predict(x={'input_1':train_xx_tensor,'cur_input_1':cur_xx_tensor})

In [ ]:
# np.save('feature.npy',feature)

In [34]:
inp, out = get_outputs_of(model,model.get_layer('time_distributed_2').output)
decoder = Model(inp, out)

In [35]:
recovered_result=decoder.predict(x=feature,steps=1)

In [36]:
np.save('test.npy',recovered_result)